In [6]:
import os
import sys
import random
from PIL import Image
import numpy as np
import shutil
import cv2
import torch

# If using a local YOLOv11 implementation, add its path (adjust as needed)
YOLOV11_PATH = "/home/abir/Desktop/Computer_Vision/Emotion/Realtime_Emotion_Detection/yolo11s-cls.pt"  # <--- set this to your local YOLOv11 code directory
sys.path.append(YOLOV11_PATH)

# Import YOLO (classification) model
from ultralytics import YOLO  # Ensure ultralytics is installed or available locally

# Define class names
class_names = ['angry', 'sad', 'happy', 'fear', 'surprise']
num_classes = len(class_names)

# Define original and processed data directories
original_data_dir = "CK+48"          # e.g., CK+48/angry/, CK+48/sad/, ...
processed_data_dir = "CK48_processed"  
train_dir = os.path.join(processed_data_dir, "train")
val_dir   = os.path.join(processed_data_dir, "val")
test_dir  = os.path.join(processed_data_dir, "test")

# Create processed data folders
for folder in [processed_data_dir, train_dir, val_dir, test_dir]:
    os.makedirs(folder, exist_ok=True)
for cls in class_names:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)


In [7]:
# Set random seed for reproducibility
random.seed(42)

# Split ratios
train_ratio = 0.8
val_ratio = 0.1
# (test_ratio implicitly 0.1 = 1 - train_ratio - val_ratio)

# Process each class folder
for cls in class_names:
    orig_cls_dir = os.path.join(original_data_dir, cls)
    images = [f for f in os.listdir(orig_cls_dir) 
              if f.lower().endswith(('.jpg','.jpeg','.png'))]
    random.shuffle(images)
    n = len(images)
    n_train = int(train_ratio * n)
    n_val = int(val_ratio * n)
    
    splits = {
        'train': images[:n_train],
        'val': images[n_train:n_train+n_val],
        'test': images[n_train+n_val:]
    }
    
    for split, img_files in splits.items():
        for img_name in img_files:
            img_path = os.path.join(orig_cls_dir, img_name)
            # Open and preprocess
            img = Image.open(img_path).convert('L').resize((48, 48))
            img = img.convert('RGB')  # convert to 3-channel (grayscale values) for YOLO model
            # Save to corresponding folder
            target_path = os.path.join(processed_data_dir, split, cls, img_name)
            img.save(target_path)


In [8]:
import yaml

data_yaml = {
    'train': train_dir,
    'val': val_dir,
    'test': test_dir,
    'nc': 5,
    'names': class_names
}
with open('data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)


In [9]:
import yaml

with open("data.yaml", "r") as f:
    yml = yaml.safe_load(f)
    print("Train path:", yml["train"])
    print("Val path:", yml["val"])


Train path: CK48_processed/train
Val path: CK48_processed/val


In [14]:
from ultralytics import YOLO

model = YOLO("yolo11s-cls.pt")  # Make sure this path is correct
results = model.train(data="/home/abir/Desktop/Computer_Vision/Emotion/Realtime_Emotion_Detection/CK48_processed", epochs=100, imgsz=48, project='runs/classify', name='emotion_exp')


Ultralytics 8.3.140 🚀 Python-3.10.0 torch-2.7.0+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/abir/Desktop/Computer_Vision/Emotion/Realtime_Emotion_Detection/CK48_processed, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=48, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=emotion_exp17, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ov

train: Scanning /home/abir/Desktop/Computer_Vision/Emotion/Realtime_Emotion_Detection/CK48_processed/train... 673 images, 0 corrupt: 100%|██████████| 673/673 [00:00<?, ?it/s]

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 37.7±24.6 MB/s, size: 1.6 KB)



/home/abir/Desktop/Computer_Vision/Emotion/Realtime_Emotion_Detection/env_emo/lib/python3.10/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning /home/abir/Desktop/Computer_Vision/Emotion/Realtime_Emotion_Detection/CK48_processed/val... 82 images, 0 corrupt: 100%|██████████| 82/82 [00:00<?, ?it/s]

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 64 train, 64 val
Using 0 dataloader workers
Logging results to runs/classify/emotion_exp17
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

    100/100         0G    0.09501          1         64: 100%|██████████| 43/43 [00:04<00:00, 10.06it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00, 25.78it/s]

                   all      0.988          1



100 epochs completed in 0.148 hours.
Optimizer stripped from runs/classify/emotion_exp17/weights/last.pt, 11.0MB
Optimizer stripped from runs/classify/emotion_exp17/weights/best.pt, 11.0MB

Validating runs/classify/emotion_exp17/weights/best.pt...
Ultralytics 8.3.140 🚀 Python-3.10.0 torch-2.7.0+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
YOLO11s-cls summary (fused): 47 layers, 5,440,533 parameters, 0 gradients, 12.0 GFLOPs
train: /home/abir/Desktop/Computer_Vision/Emotion/Realtime_Emotion_Detection/CK48_processed/train... found 673 images in 5 classes ✅ 
val: /home/abir/Desktop/Computer_Vision/Emotion/Realtime_Emotion_Detection/CK48_processed/val... found 82 images in 5 classes ✅ 
test: /home/abir/Desktop/Computer_Vision/Emotion/Realtime_Emotion_Detection/CK48_processed/test... found 88 images in 5 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00, 17.82it/s]


                   all          1          1
Speed: 0.0ms preprocess, 1.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/emotion_exp17


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Load the best model checkpoint
best_model_path = os.path.join('runs', 'classify', 'emotion_exp17', 'weights', 'best.pt')
model = YOLO(best_model_path)

y_true = []
y_pred = []

# Run through validation images
for cls in class_names:
    cls_dir = os.path.join(val_dir, cls)
    for img_name in os.listdir(cls_dir):
        img_path = os.path.join(cls_dir, img_name)
        # Predict using YOLO model
        results = model(img_path)  # returns a list of Results
        probs = results[0].probs.numpy()[0]  # probabilities for each class
        pred_class = int(np.argmax(probs))
        y_true.append(class_names.index(cls))
        y_pred.append(pred_class)

# Compute metrics
print("Validation Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Validation)')
plt.show()


ModuleNotFoundError: No module named 'expecttest'

In [ ]:
y_true = []
y_pred = []

for cls in class_names:
    cls_dir = os.path.join(test_dir, cls)
    for img_name in os.listdir(cls_dir):
        img_path = os.path.join(cls_dir, img_name)
        results = model(img_path)
        probs = results[0].probs.numpy()[0]
        pred_class = int(np.argmax(probs))
        y_true.append(class_names.index(cls))
        y_pred.append(pred_class)

print("Test Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))
cm_test = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,5))
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Greens', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Test)')
plt.show()


In [ ]:
# Initialize webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Could not open webcam")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess frame: convert to grayscale and resize
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (48, 48))
    input_img = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)  # convert back to 3-channel
    
    # Run YOLO model
    results = model(input_img)
    probs = results[0].probs.numpy()[0]
    pred_class = int(np.argmax(probs))
    label = class_names[pred_class]
    confidence = probs[pred_class]
    
    # Display label on the original frame
    cv2.putText(frame, f"{label} ({confidence:.2f})", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
